In [1]:
import pandas as pd
import random
random.seed(2)

# Read in the data
df = pd.read_csv('humans-output-new-wiki-1.csv')

In [2]:
annotater_list = list(set(df['Human 1'].unique().tolist() + df['Human 2'].unique().tolist()))

In [3]:
# create a id list for each row of df
df['id'] = df.index

cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols]

In [4]:
# create a dictionary with annotater as key and empty list as value
annotater_dict = {}
for annotater in annotater_list:
    annotater_dict[annotater] = []

# create a dictionary with id as key and empty list as value
id_dict = {}
for id in df['id'].tolist():
    id_dict[id] = []

In [5]:
# going through each row of df
for index, row in df.iterrows():
    annotater_one = row['Human 1']
    annotater_two = row['Human 2']
    remaining_annotaters = list(set(annotater_list) - set([annotater_one, annotater_two]))
    # random sampling 3 annotaters from remaining annotaters
    for annotater in remaining_annotaters:
        if len(annotater_dict[annotater]) == 15:
            remaining_annotaters.remove(annotater)
            break
    annotaters = random.sample(remaining_annotaters, 3)
    for annotater in annotaters:
        annotater_dict[annotater].append(row['id'])
        id_dict[row['id']].append(annotater)
    
    id_dict[row['id']] = annotaters

In [6]:
# print length of each annotater's list
for annotater in annotater_dict:
    print(annotater, len(annotater_dict[annotater]))

kelly 15
vinayak 15
vishnesh 12
anton 15
rachel 16
ayush 17


In [7]:
id_dict

{0: ['anton', 'vinayak', 'ayush'],
 1: ['rachel', 'vishnesh', 'anton'],
 2: ['ayush', 'vinayak', 'anton'],
 3: ['vishnesh', 'ayush', 'vinayak'],
 4: ['vinayak', 'kelly', 'anton'],
 5: ['rachel', 'ayush', 'kelly'],
 6: ['kelly', 'vishnesh', 'ayush'],
 7: ['ayush', 'rachel', 'vinayak'],
 8: ['vinayak', 'anton', 'vishnesh'],
 9: ['anton', 'vishnesh', 'kelly'],
 10: ['anton', 'vinayak', 'rachel'],
 11: ['kelly', 'vishnesh', 'vinayak'],
 12: ['kelly', 'vinayak', 'ayush'],
 13: ['ayush', 'vinayak', 'rachel'],
 14: ['kelly', 'anton', 'vishnesh'],
 15: ['ayush', 'kelly', 'anton'],
 16: ['anton', 'vinayak', 'rachel'],
 17: ['ayush', 'rachel', 'kelly'],
 18: ['vinayak', 'rachel', 'ayush'],
 19: ['vinayak', 'kelly', 'anton'],
 20: ['kelly', 'vinayak', 'vishnesh'],
 21: ['ayush', 'rachel', 'vinayak'],
 22: ['anton', 'ayush', 'vishnesh'],
 23: ['rachel', 'vishnesh', 'kelly'],
 24: ['kelly', 'rachel', 'vishnesh'],
 25: ['kelly', 'anton', 'rachel'],
 26: ['ayush', 'kelly', 'rachel'],
 27: ['ayush', '

In [8]:
id_dict[16] = ['vinayak', 'rachel', 'vishnesh']
id_dict[21] = ['ayush', 'vinayak', 'rachel']

In [9]:
# reformat annotaer_dict based on id_dict
for annotater in annotater_dict:
    annotater_dict[annotater] = []
    for id in id_dict:
        if annotater in id_dict[id]:
            annotater_dict[annotater].append(id)

In [10]:
# import json

# # save annotater_dict to json
# with open('annotater_dict-new-wiki-1.json', 'w') as fp:
#     json.dump(annotater_dict, fp)

# # save id_dict to json
# with open('id_dict-new-wiki-1.json', 'w') as fp:
#     json.dump(id_dict, fp)
import json

annotater_dict = json.load(open('annotater_dict-new-wiki-1.json'))
id_dict = json.load(open('id_dict-new-wiki-1.json'))

In [11]:
# print length of each annotater's list
for annotater in annotater_dict:
    print(annotater, len(annotater_dict[annotater]))

vinayak 15
kelly 15
rachel 15
vishnesh 15
ayush 15
anton 15


In [12]:
# random switch human 1 and human 2 and their writings
for index, row in df.iterrows():
    if random.random() > 0.5:
        df.loc[index, 'Human 1'] = row['Human 2']
        df.loc[index, 'Human 2'] = row['Human 1']
        df.loc[index, 'Human 1 Writing'] = row['Human 2 Writing']
        df.loc[index, 'Human 2 Writing'] = row['Human 1 Writing']

In [13]:
import json
with open('../gpt-3/gpt3-output.json') as f:
    gpt3_zeroshot = json.load(f)

# few-shot
with open('../gpt-3/gpt3-few-shot-output.json') as f:
    gpt3_fewshot = json.load(f)

with open("muss-output-new-wiki-1.json") as f:
    muss = json.load(f)

In [14]:
df['GPT-3-zero-shot'] = gpt3_zeroshot
df['GPT-3-few-shot'] = gpt3_fewshot
df['Muss'] = muss

In [15]:
from string import punctuation
import nltk
nltk.download('punkt')
from nltk import sent_tokenize

def remove_tokenization_artifacts(s, src):
    stokens = s.split()
    snew = s
    for i, token in enumerate(stokens):
        if i > 0 and i < len(stokens) - 1 and token in punctuation:
            substrboth = stokens[i - 1] + token + stokens[i + 1]
            substrleft = stokens[i - 1] + token
            substright = token + stokens[i + 1]
            if substrboth in src:
                snew = snew.replace(stokens[i - 1] + " " + token + " " + stokens[i + 1], substrboth)
            elif substrleft in src:
                snew = snew.replace(stokens[i - 1] + " " + token, substrleft)
            elif substright in src:
                snew = snew.replace(token + " " + stokens[i + 1], substright)

    snew = snew.replace("''", '"')
    snew = snew.replace(" .", ".")
    snew_rest = "" if len(snew) == 1 else snew[1:]
    if len(snew) > 0:
        snew = snew[0].capitalize() + snew_rest
    snew = snew.replace("-lrb-", "(").replace("-rrb-", ")")
    snew = snew.replace("-LRB-", "(").replace("-RRB-", ")")
    return snew

def reformat_output(output, original):
    output = remove_tokenization_artifacts(output, original)
    output = " || ".join(sent_tokenize(output))
    return output

[nltk_data] Downloading package punkt to /Users/yaod_1/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [16]:
# reformat output with reformat_output function
for index, row in df.iterrows():
    df.loc[index, 'Human 1 Writing'] = reformat_output(row['Human 1 Writing'], row['Original Sentence'])
    df.loc[index, 'Human 2 Writing'] = reformat_output(row['Human 2 Writing'], row['Original Sentence'])
    df.loc[index, 'GPT-3-zero-shot'] = reformat_output(row['GPT-3-zero-shot'], row['Original Sentence'])
    df.loc[index, 'GPT-3-few-shot'] = reformat_output(row['GPT-3-few-shot'], row['Original Sentence'])
    df.loc[index, 'Muss'] = reformat_output(row['Muss'], row['Original Sentence'])

In [17]:
df.to_csv('all-outputs-new-wiki-1.csv', index=False)

In [18]:
# {'vinayak': [0, 1, 3, 7, 8, 10, 11, 13, 16, 18, 19, 20, 21, 22, 23],
#  'kelly': [0, 3, 4, 5, 9, 12, 13, 14, 15, 17, 23, 24, 25, 26, 28],
#  'rachel': [1, 2, 10, 11, 14, 16, 19, 21, 23, 24, 25, 26, 27, 28, 29],
#  'vishnesh': [1, 3, 4, 5, 6, 7, 9, 11, 14, 15, 16, 17, 18, 20, 29],
#  'ayush': [2, 5, 6, 7, 8, 9, 12, 13, 17, 18, 21, 22, 26, 27, 28],
#  'anton': [0, 2, 4, 6, 8, 10, 12, 15, 19, 20, 22, 24, 25, 27, 29]}

In [19]:
annotater_dict

{'vinayak': [0, 1, 3, 7, 8, 10, 11, 13, 16, 18, 19, 20, 21, 22, 23],
 'kelly': [0, 3, 4, 5, 9, 12, 13, 14, 15, 17, 23, 24, 25, 26, 28],
 'rachel': [1, 2, 10, 11, 14, 16, 19, 21, 23, 24, 25, 26, 27, 28, 29],
 'vishnesh': [1, 3, 4, 5, 6, 7, 9, 11, 14, 15, 16, 17, 18, 20, 29],
 'ayush': [2, 5, 6, 7, 8, 9, 12, 13, 17, 18, 21, 22, 26, 27, 28],
 'anton': [0, 2, 4, 6, 8, 10, 12, 15, 19, 20, 22, 24, 25, 27, 29]}

In [20]:
import re

system_list = ["Human 1 Writing", "Human 2 Writing", "GPT-3-zero-shot", "GPT-3-few-shot", "Muss"]
for annotater in annotater_dict:
    annotater_batch = []
    start_id = 0
    for id in annotater_dict[annotater]:
        # get the id th row of df
        row = df.iloc[id]
        random.seed(2)
        # random shuffle the system list
        random.shuffle(system_list)
        for system in system_list:
            current_dict = {}
            current_dict["id"] = start_id
            current_dict["original"] = row["Original Sentence"].strip()
            current_dict["original_spans"] = []

            simplified = row[system].strip()
            simplified = re.sub(r" 's", "'s", simplified)
            current_dict["simplified"] = simplified
            current_dict["simplified_spans"] = []

            if "||" in simplified:
                #  find index of all || in simplified
                indices = [m.start() for m in re.finditer('\|\|', simplified)]
                for i, indice in enumerate(indices):
                    current_dict["simplified_spans"].append([2, indice, indice+2, i])
            
            current_dict["system"] = "new-wiki-1/" + system
            annotater_batch.append(current_dict)
            start_id += 1

    # split annotater_batch into 2 batches first 40 and last 35
    annotater_batch_1 = annotater_batch[:40]
    annotater_batch_2 = annotater_batch[40:]

    with open("../batches/new-wiki-1/part1/" + annotater + ".json", "w") as f:
        json.dump(annotater_batch_1, f, indent=4)

    # reindex annotater_batch_2's id from 0
    for i, item in enumerate(annotater_batch_2):
        item["id"] = i

    with open("../batches/new-wiki-1/part2/" + annotater + ".json", "w") as f:
        json.dump(annotater_batch_2, f, indent=4)